#HR Separation. 
Checking the annual retirement and separation rates given a model.

In [1]:
coredir = '/data/discovery/hrsepara/core/'
stgdir1 = '/data/discovery/hrsepara/staging/eda'
stgdir1local = '/home/kesj/work/hrsepara/eda'
stgdir2local = '/home/hrsepara/work'
repodir = '/home/kesj/lib/repo/'

In [2]:
### load the basic files
import os,subprocess,sys
import pandas as pd
import numpy as np
%matplotlib inline
from datetime import datetime
import matplotlib.pyplot as plt
from itertools import chain
import random
import seaborn as sns
from glob import glob
plt.style.use('ggplot')

/opt/anaconda/2.3.0/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
/opt/anaconda/2.3.0/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


## Plan of attack:
1. read the data (after 2001)
2. obtain the set of current employees
3. Run through a model to get prediction(s) for retirement and separation
    * based upon the given model, preprocess it appropriately: for the server model use the minimal set of features and processing of the web_app code.
    * for the survival model ...
4. calc the total separation and retirements for each year and derive the rate of separation/retirement.
    * how do I determine the confidence intervals for these?

In [3]:
#os.chdir(stgdir1local)
#em2002 = pd.read_csv('employees_after2001_raw.csv',dtype={'EMPL_CLASS':np.str,'LOCATION':np.str,'EEO1CODE':np.str,'SHIFT':np.str,'VOLINVOL':np.str})
#em2002.shape
##em2 = pd.read_csv('after2001_v1.csv')
##em2.head()
os.chdir('/home/hrsepara/work/data/')
em2002 = pd.read_csv('after2001_v3.csv',dtype={'EMPL_CLASS':np.str,'EEO1CODE':np.str,'KEY':np.str})
em2002.shape

(134266, 69)

In [4]:
'status' in em2002.columns.tolist()

True

In [5]:
pd.crosstab(em2002.status, em2002.retired)

retired,0,1
status,,
0,69475,0
1,49674,15117


In [6]:
## save this dataframe
#os.chdir(stgdir2local)
#em2002.to_csv('after2001_keyed.csv',index=False)

## first define the current employees

In [7]:
current = em2002[em2002.status==0].copy() 

In [8]:
np.shape(current)

(69475, 69)

## now look to apply the HRserver model (reduced column)

In [ ]:
#cols_to_keep = ['KEY','HIRE_DT','BIRTHDATE','SAL1','MERIT1','PERF1','BOX1','SEX','HAVE_INS','HAVE_DEP']
#columns_for_modeling = ['Age_years','Tenure_years','SAL1','MERIT1','PERF1','BOX1','SEX','HAVE_INS','HAVE_DEP']

In [ ]:
#employee_df = current[columns_for_modeling].copy()

In [ ]:
#employee_df.isnull().any()

# March 4, 2016
## Look at applying my current best models to the set of current employees:
* retirement_sf2.pkl 
* separation_cf4.pkl
### Load these files

In [9]:
os.chdir('/home/kesj/work/hrsepara/proc')
retirement_mdl_file = 'retirement_sf2.pkl'
separation_mdl_file = 'separation_cf4.pkl'

In [10]:
import pickle

In [11]:
ret_mdl = pickle.load(open(retirement_mdl_file))
ret_mdl.print_summary()

n=34767, number of events=11905

                coef  exp(coef)  se(coef)          z          p  lower 0.95  upper 0.95     
Age_years  3.388e-01  1.403e+00 1.010e-02  3.354e+01 1.192e-246   3.190e-01   3.586e-01  ***
SAL1      -2.439e-01  7.836e-01 1.313e-02 -1.858e+01  4.794e-77  -2.697e-01  -2.182e-01  ***
MERIT1    -2.373e-01  7.888e-01 1.416e-02 -1.675e+01  5.614e-63  -2.650e-01  -2.095e-01  ***
PERF1     -1.240e-01  8.834e-01 1.373e-02 -9.032e+00  1.686e-19  -1.509e-01  -9.708e-02  ***
BOX1      -1.031e-02  9.897e-01 1.254e-02 -8.222e-01  4.110e-01  -3.488e-02   1.427e-02     
SEX        1.697e-01  1.185e+00 1.129e-02  1.503e+01  4.522e-51   1.475e-01   1.918e-01  ***
HAVE_INS  -1.610e-01  8.513e-01 1.126e-02 -1.430e+01  2.206e-46  -1.830e-01  -1.389e-01  ***
HAVE_DEP   4.226e-03  1.004e+00 1.099e-02  3.844e-01  7.007e-01  -1.732e-02   2.578e-02     
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1 

Concordance = 0.723


In [12]:
sep_mdl = pickle.load(open(separation_mdl_file))
sep_mdl.print_summary()

n=76368, number of events=31927

                       coef  exp(coef)  se(coef)          z          p  lower 0.95  upper 0.95     
Age_years        -9.386e-01  3.912e-01 1.023e-02 -9.175e+01  0.000e+00  -9.587e-01  -9.185e-01  ***
SAL1             -1.115e+00  3.278e-01 1.479e-02 -7.539e+01  0.000e+00  -1.144e+00  -1.086e+00  ***
MERIT1            2.742e-01  1.315e+00 6.343e-03  4.323e+01  0.000e+00   2.618e-01   2.866e-01  ***
PERF1            -1.016e-01  9.034e-01 1.240e-02 -8.195e+00  2.514e-16  -1.259e-01  -7.730e-02  ***
BOX1             -4.839e-01  6.164e-01 6.824e-03 -7.090e+01  0.000e+00  -4.972e-01  -4.705e-01  ***
SEX               1.643e-01  1.179e+00 6.098e-03  2.695e+01 5.826e-160   1.524e-01   1.763e-01  ***
HAVE_INS         -3.066e-01  7.359e-01 9.072e-03 -3.380e+01 1.975e-250  -3.244e-01  -2.888e-01  ***
HAVE_DEP         -2.326e-01  7.925e-01 1.147e-02 -2.027e+01  2.258e-91  -2.550e-01  -2.101e-01  ***
DEPTCNT1         -3.471e-01  7.068e-01 9.569e-03 -3.627e+01 4.495e-

### begin with Separation

In [53]:
def get_surv_model_columns(surv_mdl):
    #event_col = surv_mdl.event_observed.name
    time_col = surv_mdl.durations.name
    model_cols = surv_mdl.hazards_.columns.tolist()
    cols_to_keep = [a for a in model_cols]
    cols_to_keep.append(event_col)
    cols_to_keep.append(time_col)
    return cols_to_keep

In [14]:
sep_cols_to_keep = get_surv_model_columns(sep_mdl)
print(sep_cols_to_keep)

['Age_years', 'SAL1', 'MERIT1', 'PERF1', 'BOX1', 'SEX', 'HAVE_INS', 'HAVE_DEP', 'DEPTCNT1', 'EXTFUNC_CNT', 'FUNC_CNT', 'REMOTE', 'DIRECT_RPT_CNT', 'HUBIND', 'REH_CNT', 'JOB_FUNCTION_PSA', 'status', 'Tenure_years']


### define columns that are NOT in the input file

In [15]:
cols_to_construct = list(set(sep_cols_to_keep)-set(current.columns.tolist()))
print(cols_to_construct)

['JOB_FUNCTION_PSA']


In [16]:
pieces = cols_to_construct[0].split('_')
print(len(pieces))
dummy_cat = pieces[-1]
main_cat = "_".join(piece for piece in pieces[:-1])
main_cat

3


'JOB_FUNCTION'

In [17]:
## transform the input to this format
current[cols_to_construct[0]]=current[main_cat].apply(lambda x: 1 if x==dummy_cat else 0)
#current[main_cat][current[main_cat]==dummy_cat].

In [18]:
employee_df = current[sep_cols_to_keep].copy()

## repeat for retirement

In [19]:
ret_cols_to_keep = get_surv_model_columns(ret_mdl)
ret_cols_to_construct = list(set(ret_cols_to_keep)-set(current.columns.tolist()))
ret_cols_to_construct

[]

In [20]:
ret_empl = current[ret_cols_to_keep].copy()

## Begin applying the models to the data

In [21]:
employee_df.isnull().any() # nothing is missing


Age_years           False
SAL1                False
MERIT1              False
PERF1               False
BOX1                False
SEX                 False
HAVE_INS            False
HAVE_DEP            False
DEPTCNT1            False
EXTFUNC_CNT         False
FUNC_CNT            False
REMOTE              False
DIRECT_RPT_CNT      False
HUBIND              False
REH_CNT             False
JOB_FUNCTION_PSA    False
status              False
Tenure_years        False
dtype: bool

In [22]:
employee_df[['SAL1','PERF1','MERIT1']].describe()

,SAL1,PERF1,MERIT1
count,69475.000000,69475.000000,69475.000000
mean,63044.836594,2193.765437,1824.702200
std,31289.533827,1898.840218,2362.811163
min,9094.800000,0.000000,-45000.000000
25%,38642.465000,877.500000,904.230000
50%,57323.480000,1928.370000,1630.710000
75%,81215.420000,3213.000000,2514.330000
max,725000.000000,56751.140000,35000.000000


In [23]:
# fix the dollar amounts
min_sal1 = 17621.76 #(based upon training set I have: 5 %tile cut off)
min_min_rt_ann = 17900. # same as above
min_max_rt_ann = 33155.70

max_max_rt_ann = 133068.91
min_merit1 = 0.0
min_perf1 = 0.0
fix_min_outlier_col_dict = {'SAL1': min_sal1, 'MERIT1': min_merit1, 'PERF1': min_perf1}
#fix_max_outlier_col_dict = {'MAX_RT_ANNUAL': max_max_rt_ann}

# replace these values
for key,value in fix_min_outlier_col_dict.iteritems():        
    idx_to_replace = employee_df[employee_df[key]<value].index
    employee_df.loc[idx_to_replace,key]=value

    # dummy empl_type categories
    #empl_type_dummy_cat = ['EMPL_TYPE_E', 'EMPL_TYPE_H', 'EMPL_TYPE_N', 'EMPL_TYPE_S', 'EMPL_TYPE_X', 'EMPL_TYPE_nan']


In [24]:
employee_df[['SAL1','PERF1','MERIT1']].describe()

,SAL1,PERF1,MERIT1
count,69475.000000,69475.000000,69475.000000
mean,63098.086224,2193.765437,1918.218172
std,31209.347146,1898.840218,1790.079567
min,17621.760000,0.000000,0.000000
25%,38642.465000,877.500000,904.230000
50%,57323.480000,1928.370000,1630.710000
75%,81215.420000,3213.000000,2514.330000
max,725000.000000,56751.140000,35000.000000


## Oh no these versions of the fitter don't have my utilities associated with them....

In [25]:
## Forecast forward in 1 year intervals
def identify_forecast_timepoints(mdl, X, time_range):
    """
        :param X: a (n,d) covariate numpy array or DataFrame. If a numpy array, it is coerced into a DataFrame
        :param time_range: a list of times to calculate the survival for.
        :return: time_point_df a DataFrame of selected future times to create survival forecasts for.

        Construct a data frame that has the current time_col for each observation incremented by the values in the
        desired list (time_range)
    """
    time_col = mdl.durations.name
    column_names = ['time_point_'+np.str(tp) for tp in time_range]
    if isinstance(X, pd.DataFrame):
        my_index = X.index.tolist()
    else:
        my_index = np.arange(0, len(X))
        X = pd.DataFrame(X, columns=mdl.hazards_.columns)

    time_point_df = pd.DataFrame(columns=column_names, index=my_index)
    for idx in my_index:
        my_times = np.array([X.loc[idx, time_col]+tp for tp in time_range])
        time_point_df.ix[idx] = my_times

    return time_point_df

In [32]:
from scipy import interpolate
def return_desired_cumulative_hazards(t,mdl):
    """
    t: an event time that is used as the base line to forecast forward in time
    """
    maxtimepoint = mdl.baseline_hazard_.index.max() #Now taken care of in the forecast
    if t > maxtimepoint:
        t = maxtimepoint # just set to the max value.
    try:
        # spec_hazard = self.baseline_hazard_.ix[t].values
        c_haz = mdl.baseline_cumulative_hazard_.ix[t].values[0]

    except KeyError:
        # get the first after this point
        t_post = mdl.baseline_hazard_.ix[t:].index[0]
        # get the last before this point
        t_prior = mdl.baseline_hazard_.ix[:t].index[-1]
        # x = [t_prior, t_post]
        y = [mdl.baseline_cumulative_hazard_.ix[t_prior].values[0],
             mdl.baseline_cumulative_hazard_.ix[t_post].values[0]]
        # print(x, y)
        chaz_interp = interpolate.interp1d([t_prior, t_post], y)
        c_haz = chaz_interp(t)

    return c_haz

In [ ]:
#sep_timepoints = identify_forecast_timepoints(sep_mdl,employee_df,[0,1,2,3,4,5])

In [27]:
from scipy.optimize import curve_fit

def line(x, a, b):
    return a*x+b


In [36]:
def forecast_survival_function(mdl, df, time_range=[0,1,2,3,4,5], calc_ci=True, extrapolate=False):
    """
        :param self:
        :param df:

        :param time_range:
        :param calc_ci: flag to return the confidence intervals
        :return:
    """

    #time_col = self.durations.name
    col_names = ['t_'+np.str(tp) for tp in time_range]
    # calculate the predicted hazard
    my_partial_hazard = mdl.predict_partial_hazard(df)
    # assign the specific timepoints and calculate the cumulative hazards at these points
    forecast_times = identify_forecast_timepoints(mdl,df, time_range)
    max_time = mdl.baseline_hazard_.index.max()
    if extrapolate:
        # do a linear extrapolation of the cumulative hazards and use this to get values beyond the known range
        p_haz, p_haz_cov = curve_fit(line, mdl.baseline_cumulative_hazard_.index,
                                 mdl.baseline_cumulative_hazard_.values.ravel())
            # force the points to match
        offset = mdl.baseline_cumulative_hazard_.values[-1][0] - line(mdl.baseline_cumulative_hazard_.index[-1],
                                                                           p_haz[0] ,p_haz[1])
            # apply to timepoints
        specific_cumulative_hazards = forecast_times.applymap(lambda x: return_desired_cumulative_hazards(mdl,x)
            if x <= max_time else offset+line(x,popt[0],popt[1]))
    else:  # this is the default case
            # replace those that exceed the max-time observed with the max observed time
        forecast_times[forecast_times > max_time] = max_time
        specific_cumulative_hazards = forecast_times.applymap(lambda x: return_desired_cumulative_hazards(x,mdl))

        # predict the survival function at these forecasted times
        pred_survival_fcn = pd.DataFrame(np.exp(-np.multiply(specific_cumulative_hazards.values,
                                                             my_partial_hazard.values)), index=df.index)

    if calc_ci:
        # calculate the predicted hazard
        partial_hazard_ci = mdl.predict_partial_hazard_ci(df)
        pred_lower_survival_ci = pd.DataFrame(np.exp(-np.multiply(specific_cumulative_hazards.values.T,
                                                                  partial_hazard_ci.values[:, 1])),
                                              columns=df.index).T
        pred_upper_survival_ci = pd.DataFrame(np.exp(-np.multiply(specific_cumulative_hazards.values.T,
                                                                  partial_hazard_ci.values[:, 0])),
                                              columns=df.index).T

        surv_prediction = pd.Panel(data={'surv': pred_survival_fcn, 'lbci': pred_lower_survival_ci,
                                         'ubci': pred_upper_survival_ci})
    else:
        surv_prediction = pd.Panel(data={'surv': pred_survival_fcn})
                                   #, 'lbci': pred_survival_fcn,
                                   #      'ubci': pred_survival_fcn})
    surv_prediction.minor_axis = col_names

    return surv_prediction

## lets break this down

In [38]:
%%time
tdp = identify_forecast_timepoints(sep_mdl, employee_df, time_range =[0,1,2,3,4,5])

CPU times: user 25min 11s, sys: 473 ms, total: 25min 12s
Wall time: 25min 11s


In [39]:
#%%time
#sep_forecast = forecast_survival_function(sep_mdl, employee_df, time_range=[0,1,2,3,4,5], calc_ci=True, extrapolate=False)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 17.2 µs


In [40]:
my_partial_hazard = sep_mdl.predict_partial_hazard(employee_df)

In [41]:
max_time = sep_mdl.baseline_hazard_.index.max()
print(max_time)

58.0408906411


In [42]:
tdp2 = tdp.copy()
tdp2[tdp2>max_time]=max_time # just set to the max value (no extrapolation)

In [44]:
spec_cum_haz = tdp2.applymap(lambda x: return_desired_cumulative_hazards(x,sep_mdl))

In [45]:
sep_pred_survival_function = pd.DataFrame(np.exp(-np.multiply(spec_cum_haz.values,
                                                             my_partial_hazard.values)), index=employee_df.index)

In [48]:
sep_proba = 1-sep_pred_survival_function
sep_proba['KEY']= current.KEY

In [49]:
sep_proba.columns = ['sep_prob0','sep_prob1','sep_prob2','sep_prob3','sep_prob4','sep_prob5','KEY']
sep_proba.head()

,sep_prob0,sep_prob1,sep_prob2,sep_prob3,sep_prob4,sep_prob5,KEY
64791,0.416907,0.433157,0.456523,0.469564,0.495995,0.516903,929965184037557
64792,0.411617,0.430857,0.450175,0.466659,0.492508,0.505002,2396741883083
64793,0.092980,0.098332,0.104625,0.109815,0.115951,0.122274,872454415016088
64794,0.126733,0.136192,0.147030,0.159128,0.173263,0.188095,247046588569436
64795,0.130791,0.142563,0.153719,0.163980,0.173378,0.181629,797202090638318


In [50]:
sep_proba = sep_proba[['KEY','sep_prob0','sep_prob1','sep_prob2','sep_prob3','sep_prob4','sep_prob5']]
sep_proba.head()

,KEY,sep_prob0,sep_prob1,sep_prob2,sep_prob3,sep_prob4,sep_prob5
64791,929965184037557,0.416907,0.433157,0.456523,0.469564,0.495995,0.516903
64792,2396741883083,0.411617,0.430857,0.450175,0.466659,0.492508,0.505002
64793,872454415016088,0.092980,0.098332,0.104625,0.109815,0.115951,0.122274
64794,247046588569436,0.126733,0.136192,0.147030,0.159128,0.173263,0.188095
64795,797202090638318,0.130791,0.142563,0.153719,0.163980,0.173378,0.181629


In [51]:
sep_proba.to_csv('current_sep_prob.csv',index=False)

In [56]:
(sep_proba.sep_prob0>=0.5).astype(int).sum()

2534

In [52]:
%pwd

u'/home/kesj/work/hrsepara/proc'

## for retirement I need to predict prob = 0 if age <50.

## Load my version of lifelines

In [ ]:
ll_repo_dir = '/home/kesj/lib/repo/lifelines'
os.chdir(ll_repo_dir)
import lifelines as ll

In [ ]:
sep_mdl.

In [ ]:
# get the current date
import datetime
today_timestamp = pd.to_datetime(datetime.datetime.today())
print today_timestamp
# replace with Jan 01-15
today_timestamp =pd.to_datetime(datetime.datetime.strptime('01JAN2015','%d%b%Y'))
print today_timestamp

In [ ]:
def calculate_years(timestamp1,timestamp2,days_in_year =365.25):
    number_of_years = (timestamp2 - timestamp1).days/days_in_year
    return number_of_years

In [ ]:
## create timestamps for columns that are timestamps

employee_df[['hire_tstmp','birth_tstmp']] = employee_df[['HIRE_DT','BIRTHDATE']].applymap(lambda x: pd.to_datetime(datetime.datetime.strptime(x,'%d%b%Y')))
employee_df[['Age_years','Tenure_years']] = employee_df[['birth_tstmp','hire_tstmp']].applymap(lambda x: calculate_years(x,today_timestamp))

# now deal with the the rest of the columns (map using the above  )
columns_for_modeling = ['Age_years','Tenure_years','SAL1','MERIT1','PERF1','BOX1','SEX','HAVE_INS','HAVE_DEP',]
    #'EMPL_TYPE']
# replace Y with 1 and N with 0
# replace M with 1 and F with 0 (in SEX)


In [ ]:
empl=employee_df[columns_for_modeling].copy()
empl.fillna({'HAVE_INS':'N','HAVE_DEP':'N','BOX1':0,'MERIT1':0.0,'PERF1':0.0,'SAL1':0.0},inplace=True)
empl.isnull().any()


In [ ]:
## replace specific elements based upon dictionary
replace_dict = {'HAVE_INS': {'Y':1,'N':0}, 'HAVE_DEP': {'Y':1,'N':0}, 'BOX1':{'H':3,'S':2,'L':1},'SEX':{'M':1,'F':0}}
for key,value in replace_dict.iteritems():
    empl[key].replace(value,inplace=True)


In [ ]:
X = employee_df.as_matrix()

In [ ]:
np.shape(X)

In [ ]:
# define the application of a series of time-based model
def apply_time_models(X,MODEL_BASE_PATH,pcase,pred_years):
    proba_df = []


    import joblib as jl
    import pandas as pd

    for idx,pyr in enumerate(pred_years):
        pkl_name ='rf'+pyr+'.pkl'#5.pkl'
        if pcase == 'ret/':
            pkl_name = 'R'+pkl_name

        path_name =MODEL_BASE_PATH+pcase+pyr+'/'
        print path_name, pkl_name

        # load a model, evaluate it and return the 'average' probability for each person
        #abs_mdl_name = os.path.abspath(path_name+pkl_name)
        mdl_name = path_name+pkl_name
        stored_mdl = jl.load(mdl_name)
        stored_prediction = evaluate_models(stored_mdl,X)#stored_pred,stored_pred_proba = evaluate_models(stored_mdl,X)

        df = pd.DataFrame(stored_prediction.T)
        df.columns=[pcase[:-1]+pyr+'yr']

        proba_df.append(df)

    pred_df = pd.concat([a for a in proba_df], axis=1)
    return pred_df

# define a function to apply a set of models to a given prediction
## now apply each model to my eval set
def evaluate_models(model_list,X,mode='mean',offset=0):
    """ Function to apply a set of models to a given input and generate the predicted value(s)
    :param model_list --> input list of models
    :param X --> input array to apply models to
    :param mode --> what sort of output to return; default is mean
    intermediates
        eval_pred_class --> array of classification prediction for each model
        eval_pred_proba --> array of predicted probabilities for each model
    """
    import numpy as np
    eval_pred_class = np.zeros((len(X),len(model_list)))
    eval_pred_proba = np.zeros((len(X),2,len(model_list)))

    for i,mdl in enumerate(model_list):
        eval_proba = mdl.predict_proba(X)
        eval_pred_class[:,i]=mdl.predict(X)
        eval_pred_proba[:,:,i]=eval_proba
    if mode == 'mean':
        # average the probabilities (for class=1) and return the mean predicted probability
        prediction = np.mean(eval_pred_proba[:,1,:],axis=1)
        #eval_pred_proba[:,1,:].mean(axis=2)

    elif mode == 'class': # return the desired class prediction
        prediction = map(np.int,eval_pred_class.mean(axis=1))
    return prediction


In [ ]:
# load the model
MODEL_BASE_PATH = '/home/hrsepara/work/model/'
#MODEL_BASE_PATH = 'C:\\Users\\kesj\\code\\hrsep1\\model_1\\'
pred_cases = ['sep/','ret/']
pred_years = map(np.str,np.arange(1,6))

sep_proba_df = apply_time_models(X,MODEL_BASE_PATH,pred_cases[0],pred_years)
ret_proba_df = apply_time_models(X,MODEL_BASE_PATH,pred_cases[1],pred_years)
#sep_proba_df.index=employee_df.KEY
#ret_proba_df.index=employee_df.KEY

In [ ]:
len(sep_proba_df), len(ret_proba_df), len(employee_df)

## Feb 9, 2016 Assess these predictions
* for separation:
    1. calculate total number of separations in next 5 years
    2. calculate total number for each of the specific jobGrADE, etc. listed in previous email.
    
* for retirement:
    1. calculate total number of retirements (restrict to those overX)
    2. compare to sf model


### SEPARATION ASSESSMENT

####Global

### Set up a dictionary for some of these 'interesting' jobs
#### December 31, 2015

* The extended Functions (EXT_FUNC_ID_SFI) Jeff Briks is interested in are:
    * CCC Sales (00892)
    * CCC Service (00071)
    * Life Health Response Center (00816)
    * Enterprise Auto Express (00909)
    * Enterprise ILR (00910)
    * State Farm Bank (00517)
    * SFPP (00835)
* The jobs (JOBCODES) he is interested in are these:
    * CCC Service Rep (6200)
    * CCC Sales Rep (1876)
    * Life Customer Service Asst (2638, 2639, & 2641)
    * Claim Associate-Express (0983 & 0984)
    * Claim Associate-ILR (1009)
    * Claim Team Manager – ILR (0986)
    * Bank Customer Service Rep (0851, 852, & 0853)
    * SFPP Account Rep (4105)



In [ ]:
#interesting_extended_functions = [892,71, 816, 909,910,517,835]
extFuncDict ={}
extFuncDict[892]='CCC Sales'
extFuncDict[71]='CCC Service'
extFuncDict[816]='Life Health Response Center'
#(00816)
extFuncDict[909]='Enterprise Auto Express'
extFuncDict[910]='Enterprise ILR' 
extFuncDict[517]='State Farm Bank'
extFuncDict[835]='SFPP'
interesting_extended_functions = extFuncDict.keys()
extFuncDict

In [ ]:
jobCodeDict = {}
jobCodeDict[6200]= 'CCC Service Rep'
jobCodeDict[1876]= 'CCC Sales Rep'
jobCodeDict[1009]='CLaim Associate-ILR'
jobCodeDict[986]='CLaim Team Manager-ILR'
jobCodeDict[983]='Claim Associate-Express'
jobCodeDict[984]='Claim Associate-Express'
jobCodeDict[2638]='Life Customer Service Asst'
jobCodeDict[2639]='Life Customer Service Asst'
jobCodeDict[2641]='Life Customer Service Asst'
jobCodeDict[4105]= 'SFPP Account Rep'
jobCodeDict[851]= 'Bank Customer Service Rep'
jobCodeDict[852]= 'Bank Customer Service Rep'
jobCodeDict[853]= 'Bank Customer Service Rep'
interesting_jobcodes =  jobCodeDict.keys()
jobCodeDict

In [ ]:
print len(sep_proba_df)
sep_proba_df.tail()

In [ ]:
print len(current) 
ctemp = current[['KEY','Age_years','Tenure_years','JOBCODE','EXT_FUNC_ID_SFI']].copy()
ctemp.index =np.arange(0,len(ctemp))
ctemp.head()


In [ ]:
sep_df = pd.concat([ctemp,sep_proba_df],axis=1)
print len(sep_df)

In [ ]:
sep_df.head()

#### Global Counts of Separation:
* Method 1: sum : `sep_proba_df.sum() ` 
* Method 2: over threshhold

In [ ]:
map(np.int, sep_proba_df.sum())

In [ ]:
thresh = 0.5
sep_proba_df.applymap(lambda x: 1 if x>thresh else 0).sum()

### Assess the model validity using the testing set (from 10/6/15)


In [ ]:
em2eval = pd.read_csv('/home/kesj/work/hrsepara/proc/trans_train.csv')
[col for col in em2eval.columns.tolist() if col.endswith('DT')]

In [ ]:
[col for col in em2eval.columns.tolist() if col.endswith('tstmp')]

### okay re-form the split to get the training set

In [ ]:
print em2002.shape
from sklearn.cross_validation import train_test_split
print "Starting with subest of {0} employees.".format(len(em2002))
eval_fraction = 0.20
em2build, em2eval = train_test_split(em2002,test_size=eval_fraction,random_state=83221)
print "Evaluation set has {0} employees; training set has {1} employees.".format(len(em2eval),len(em2build))

### Now try to evaluate the mdls on em2eval

In [ ]:
em2eval_df = em2eval[columns_for_modeling].copy()
em2eval_df.isnull().any()

In [ ]:
# fix the dollar amounts
min_sal1 = 17621.76 #(based upon training set I have: 5 %tile cut off)
min_min_rt_ann = 17900. # same as above
min_max_rt_ann = 33155.70

max_max_rt_ann = 133068.91
min_merit1 = 0.0
min_perf1 = 0.0
fix_min_outlier_col_dict = {'SAL1': min_sal1, 'MERIT1': min_merit1, 'PERF1': min_perf1}
fix_max_outlier_col_dict = {'MAX_RT_ANNUAL': max_max_rt_ann}

# replace these values
for key,value in fix_min_outlier_col_dict.iteritems():        
    idx_to_replace = em2eval_df[em2eval_df[key]<value].index
    em2eval_df.loc[idx_to_replace,key]=value



In [ ]:
#sum(em2eval_df.SAL1 != em2eval.SAL1)

In [ ]:
em2eval_df['birth_tstmp']= em2eval['birth_tstmp'].apply(lambda x: pd.to_datetime(x))
em2eval_df['hire_tstmp']= em2eval['hire_tstmp'].apply(lambda x: pd.to_datetime(x))
em2eval_df['status']=em2eval['status']
em2eval_df['Age_years']=em2eval['Age_years']
em2eval_df['Tenure_years']=em2eval['Tenure_years']


In [ ]:
def adjust_eval_by_x_years(df,year_val,modeling_columns,tstmp_cols=['birth_tstmp','hire_tstmp'],target_col='status'):
    # construct
  
    ## set up method to assess the eval set
    print "There are {0} elements in the evaluation set".format(len(df))
   
    print "original target variable value counts:", df[target_col].value_counts()
    # restructure to deal with time_frame retirement (target variable)
    yr_cut_val = year_val+0.5
    # index of those that actually accomplish target within timeframe (allow 0.5 additional years)
    eval_within_time_target_index = df[(df[target_col]==1) & (df.Tenure_years <= yr_cut_val)].index
    # exclude indices that are active and have tenure less than this time
    eval_excluded_index = df[(df[target_col]==0) & (df.Tenure_years  <= yr_cut_val)].index
    
    # the rest become my not-terminated set
    eval_active_index = set(df.index) - set(eval_within_time_target_index) - set(eval_excluded_index)
    print len(eval_excluded_index),len(eval_within_time_target_index), len(eval_active_index)
    eval_idx_to_use =df.ix[set(df.index)-set(eval_excluded_index)].index
    #len(eval_idx_to_use)
    # reset the target to 0 for active
    eval_new_target = df[target_col].copy()
    eval_new_target.ix[eval_active_index] = 0
    print "new target variable value counts: "
    print eval_new_target.ix[eval_idx_to_use].value_counts()
    print "_____"
    y_eval = eval_new_target.ix[eval_idx_to_use].as_matrix().astype(np.int) # true values
    eval_adj_tenure = df.ix[eval_idx_to_use].Tenure_years.apply(lambda x: x-year_val if (x>float(year_val)) else 0).values
    print len(eval_adj_tenure), len(y_eval)
    # now adjust age by length of time; use hire_age if not in set to use.
    eval_adj_age = df.ix[eval_idx_to_use].Age_years.apply(lambda x: x-year_val)
    hire_age = (df.ix[eval_within_time_target_index]['birth_tstmp']-df.ix[eval_within_time_target_index]['hire_tstmp'])/np.timedelta64(1,'Y')#.days/days_in_year
    eval_adj_age.ix[eval_within_time_target_index] = hire_age #df.ix[eval_within_time_target_index]['birth_tstmp'#df_dates['hire_age']
    
    # construct the evaluation X matrix
    print "input matrix has {0} features".format(len(modeling_columns))
    Xeval = np.zeros((len(eval_idx_to_use),len(modeling_columns)))
    # drop 'Age_years' and Tenure_years from the list
    cols_to_use = []
    cols_to_use+=modeling_columns
    cols_to_use.remove('Age_years')
    cols_to_use.remove('Tenure_years')#.copy()
    """
    Xeval[:,:-2] = df.ix[eval_idx_to_use][cols_to_use].as_matrix().astype(np.float)
    # now put the adjusted tenure and ages into this matrix
    Xeval[:,-2] = eval_adj_age.values
    Xeval[:,-1]=eval_adj_tenure
    #print len(modeling_columns),np.shape(Xeval)
    """
    # this version matches the changed ording of columns
    Xeval[:,0]=eval_adj_age.values
    Xeval[:,1]=eval_adj_tenure
    Xeval[:,2:] = df.ix[eval_idx_to_use][cols_to_use].as_matrix().astype(np.float)
    return Xeval, y_eval

In [ ]:
Xe5,y5 =adjust_eval_by_x_years(em2eval_df,5,columns_for_modeling)

In [ ]:
MODEL_BASE_PATH = '/home/hrsepara/work/model/'
#MODEL_BASE_PATH = 'C:\\Users\\kesj\\code\\hrsep1\\model_1\\'
pred_cases = ['sep/','ret/']
pred_years = map(np.str,np.arange(1,6))

#eval_sep_proba_df = apply_time_models(Xe5,MODEL_BASE_PATH,pred_cases[0],pred_years)
eval_sep_proba5_df = apply_time_models(Xe5,MODEL_BASE_PATH,pred_cases[0],['5'])

In [ ]:
len(eval_sep_proba5_df)

In [ ]:
etemp = em2eval[['KEY','Age_years','Tenure_years','JOBCODE','EXT_FUNC_ID_SFI','status']].copy()
etemp.index =np.arange(0,len(etemp))
#etemp.head()


In [ ]:
esep_df5 = pd.concat([etemp,eval_sep_proba5_df],axis=1)
print len(esep_df5), len(y5)

In [ ]:
from sklearn import metrics
def plot_conf_matrix(y_true,y_pred,normed=True,**kwargs):
    my_c = metrics.confusion_matrix(y_true,y_pred)
    
    print metrics.matthews_corrcoef(y_true,y_pred)
    if normed:
        cm_normalized = my_c.astype('float') / my_c.sum(axis=1)[:, np.newaxis]
        my_c = cm_normalized
        plt.title('Normalized RF Classifier Confusion Matrix')
    else:
        plt.title('Random Forest Classifier Confusion Matrix')
        
    sns.heatmap(my_c, annot=True,  fmt='',cmap='Blues')
    plt.ylabel('True')
    #plt.yticks
    plt.xlabel('Assigned')
    plt.show()
    
    return

def plot_roc_curve2(target_test, target_predicted_proba, **kwargs):
    fpr, tpr, thresholds = metrics.roc_curve(target_test, target_predicted_proba)
    
    roc_auc = metrics.auc(fpr, tpr)
    # Plot ROC curve
    label_txt = 'ROC curve (area = %0.3f)' % roc_auc
    plt.plot(fpr, tpr, label=label_txt,**kwargs)
    plt.plot([0, 1], [0, 1], 'k--')  # random predictions curve
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.0])
    plt.xlabel('False Positive Rate or (1 - Specifity)')
    plt.ylabel('True Positive Rate or (Sensitivity)')
    plt.title('Receiver Operating Characteristic')
    plt.legend(loc="lower right")
    return roc_auc

In [ ]:
plot_roc_curve2(y5,eval_sep_proba5_df.sep5yr)

In [ ]:
fpr, tpr, thresholds = metrics.roc_curve(y5, eval_sep_proba5_df.sep5yr)

In [ ]:
plt.plot(tpr - fpr,'d')

In [ ]:
mydiff = tpr-fpr
max(mydiff)
#thresholds[]

In [ ]:
thresholds[np.where(mydiff == max(mydiff))]

In [ ]:
plot_conf_matrix(y5,np.map())

In [ ]:
plt.scatter(fpr, tpr)
plt.scatter(fpr[np.where(tpr>=0.99)],tpr[np.where(tpr>=0.99)],color='indianred')

In [ ]:
thresholds[np.where(tpr >= 0.99)]

In [ ]:
(eval_sep_proba5_df.sep5yr>thresh).values.astype(int)

In [ ]:
eval_sep_proba5_df.hist(bins=30)

In [ ]:
eval_sep_proba4_df.hist(bins=30)

In [ ]:
eval_sep_proba3_df.hist(bins=30)

In [ ]:
eval_sep_proba2_df.hist(bins=30)

In [ ]:
eval_sep_proba1_df.hist(bins=30)

In [ ]:
mcc_scores = []
mcc_offsets = []
for offset in np.arange(0,10)/float(10):
    #offset+=0.6
    score = metrics.matthews_corrcoef(y5,map(np.int,eval_sep_proba5_df.sep5yr+offset))
    print offset,":: ", score
    mcc_scores.append(score)
    mcc_offsets.append(offset)
    
plt.scatter(mcc_offsets,mcc_scores)

In [ ]:
mcc_offsets[mcc_scores.index(max(mcc_scores))]

In [ ]:
mcc_5_scores = []
mcc_5_offsets = []
for offset in np.arange(0,20)/float(100):
    offset+=0.6
    score = metrics.matthews_corrcoef(y5,map(np.int,eval_sep_proba5_df.sep5yr+offset))
    print offset,":: ", score
    mcc_5_scores.append(score)
    mcc_5_offsets.append(offset)
    
plt.scatter(mcc_5_offsets,mcc_5_scores)

In [ ]:
plot_conf_matrix(y5,map(np.int,eval_sep_proba5_df.sep5yr+0.69),normed=False)#gb_mdl5_pred_class.mean(axis=1)))

In [ ]:
ytest1 = eval_sep_proba5_df.sep5yr
ytest2 = map(np.int,eval_sep_proba5_df.sep5yr)
ytest3 = eval_sep_proba5_df.sep5yr.astype(int)

In [ ]:
my_type = type(ytest2)
print my_type

In [ ]:
#len(y4)
r5= [x/100000. for x in random.sample(range(0,100000),len(y5))]#/10000.

In [ ]:
def plot_conf_matrix(y_true,y_pred,normed=True, heatmap_color ='Blues', **kwargs):

    ## check to make sure that y_pred is an array of integers if y_true is a bunch of integers
    true_int_check = all(isinstance(a,int) for a in y_true)
    pred_int_check = all(isinstance(a,int) for a in y_pred)
    if true_int_check and not pred_int_check: # convert the y_pred values to integers
        if isinstance(y_pred,pd.Series):
            y_pred = y_pred.astype(int)

    my_c = metrics.confusion_matrix(y_true,y_pred)

    print metrics.matthews_corrcoef(y_true,y_pred)
    if normed:
        cm_normalized = my_c.astype('float') / my_c.sum(axis=1)[:, np.newaxis]
        my_c = cm_normalized
        plt.title('Normalized RF Classifier Confusion Matrix')
    else:
        plt.title('Random Forest Classifier Confusion Matrix')

    sns.heatmap(my_c, annot=True,  fmt='',cmap=heatmap_color)
    plt.ylabel('True')
    plt.xlabel('Assigned')
    plt.show()

    return

In [ ]:
plot_conf_matrix(y5,ytest1+0.3,normed=False)

In [ ]:
plot_conf_matrix(y5,map(np.int,eval_sep_proba5_df.sep5yr+0.3))

In [ ]:
isinstance(ytest1,list),isinstance(ytest2,list)

In [ ]:
(eval_sep_proba5_df.sep5yr+0.3)

In [ ]:
off5 = 0.69
(eval_sep_proba5_df.sep5yr+off5).values.astype(int).sum()

In [ ]:
(sep_df.sep5yr+off5).values.astype(int).sum()

## REPEAT for 1,2,3,4 years

In [ ]:
Xe4,y4 =adjust_eval_by_x_years(em2eval_df,4,columns_for_modeling)
print len(Xe4)

In [ ]:
MODEL_BASE_PATH = '/home/hrsepara/work/model/'
#MODEL_BASE_PATH = 'C:\\Users\\kesj\\code\\hrsep1\\model_1\\'
pred_cases = ['sep/','ret/']
pred_years = map(np.str,np.arange(1,6))

#eval_sep_proba_df = apply_time_models(Xe5,MODEL_BASE_PATH,pred_cases[0],pred_years)
eval_sep_proba4_df = apply_time_models(Xe4,MODEL_BASE_PATH,pred_cases[0],['4'])

In [ ]:
plot_roc_curve2(y4,eval_sep_proba4_df.sep4yr)

In [54]:
sep_mdl.print_summary()

n=76368, number of events=31927

                       coef  exp(coef)  se(coef)          z          p  lower 0.95  upper 0.95     
Age_years        -9.386e-01  3.912e-01 1.023e-02 -9.175e+01  0.000e+00  -9.587e-01  -9.185e-01  ***
SAL1             -1.115e+00  3.278e-01 1.479e-02 -7.539e+01  0.000e+00  -1.144e+00  -1.086e+00  ***
MERIT1            2.742e-01  1.315e+00 6.343e-03  4.323e+01  0.000e+00   2.618e-01   2.866e-01  ***
PERF1            -1.016e-01  9.034e-01 1.240e-02 -8.195e+00  2.514e-16  -1.259e-01  -7.730e-02  ***
BOX1             -4.839e-01  6.164e-01 6.824e-03 -7.090e+01  0.000e+00  -4.972e-01  -4.705e-01  ***
SEX               1.643e-01  1.179e+00 6.098e-03  2.695e+01 5.826e-160   1.524e-01   1.763e-01  ***
HAVE_INS         -3.066e-01  7.359e-01 9.072e-03 -3.380e+01 1.975e-250  -3.244e-01  -2.888e-01  ***
HAVE_DEP         -2.326e-01  7.925e-01 1.147e-02 -2.027e+01  2.258e-91  -2.550e-01  -2.101e-01  ***
DEPTCNT1         -3.471e-01  7.068e-01 9.569e-03 -3.627e+01 4.495e-

In [ ]:
len(eval_sep_proba4_df)

In [ ]:
def find_max_f1_value(ytrue,ypred):
    scores = []
    offsets = []
    # stage 1:
    for offset in np.arange(0,10)/float(10):
        score = metrics.f1_score(ytrue,map(np.int,ypred+offset))
        scores.append(score)
        offsets.append(offset)
    # find the max
    mxscore = max(scores)
    mx_idx = scores.index(mxscore)
    #now offset below and above
    if mx_idx == 0:
        for offset in np.arange(0,16)/float(100):
            offset+=0.0
            score = metrics.f1_score(ytrue,map(np.int,ypred+offset))
            scores.append(score)
            offsets.append(offset)
    elif mx_idx == 9:
        for offset in np.arange(0,21)/float(100):
            offset+=0.8
            score = metrics.f1_score(ytrue,map(np.int,ypred+offset))
            scores.append(score)
            offsets.append(offset)
    else:
        base_offset = offsets[mx_idx-1]
        for offset in np.arange(0,21)/float(100):
            offset+=base_offset
            score = metrics.f1_score(ytrue,map(np.int,ypred+offset))
            scores.append(score)
            offsets.append(offset)
            
    #if m
    #for offset in np.arange(0,20)/float(100):
    #    offset+=0.6
    #
    #print offset,":: ", score
    #
    #mcc_5_offsets.append(offset)
    plt.scatter(offsets,scores,color='midnightblue')
    plt.xlabel('offset value')
    plt.ylabel('F1 score')
    
    mxmxscore = max(scores)
    mxmx_idx = scores.index(mxmxscore)
    print mxmxscore, offsets[mxmx_idx]
    #return mcc_scores, mcc_offsets
    return offsets[mxmx_idx]


In [ ]:
def find_max_mcc_value(ytrue,ypred):
    mcc_scores = []
    mcc_offsets = []
    # stage 1:
    for offset in np.arange(0,10)/float(10):
        score = metrics.matthews_corrcoef(ytrue,map(np.int,ypred+offset))
        mcc_scores.append(score)
        mcc_offsets.append(offset)
    # find the max
    mxscore = max(mcc_scores)
    mx_idx = mcc_scores.index(mxscore)
    #now offset below and above
    if mx_idx == 0:
        for offset in np.arange(0,11)/float(100):
            offset+=0.0
            score = metrics.matthews_corrcoef(ytrue,map(np.int,ypred+offset))
            mcc_scores.append(score)
            mcc_offsets.append(offset)
    elif mx_idx == 9:
        for offset in np.arange(0,11)/float(100):
            offset+=0.9
            score = metrics.matthews_corrcoef(ytrue,map(np.int,ypred+offset))
            mcc_scores.append(score)
            mcc_offsets.append(offset)
    else:
        base_offset = mcc_offsets[mx_idx-1]
        for offset in np.arange(0,21)/float(100):
            offset+=base_offset
            score = metrics.matthews_corrcoef(ytrue,map(np.int,ypred+offset))
            mcc_scores.append(score)
            mcc_offsets.append(offset)
            
    #if m
    #for offset in np.arange(0,20)/float(100):
    #    offset+=0.6
    #
    #print offset,":: ", score
    #
    #mcc_5_offsets.append(offset)
    plt.scatter(mcc_offsets,mcc_scores)
    plt.xlabel('offset value')
    plt.ylabel('mcc score')
    
    mxmxscore = max(mcc_scores)
    mxmx_idx = mcc_scores.index(mxmxscore)
    print mxmxscore, mcc_offsets[mxmx_idx]
    #return mcc_scores, mcc_offsets
    return mcc_offsets[mxmx_idx]


In [ ]:
off4 = find_max_mcc_value(y4,eval_sep_proba4_df.sep4yr)

In [ ]:
off4f = find_max_f1_value(y4,eval_sep_proba4_df.sep4yr)

In [ ]:
off5f = find_max_f1_value(y5,eval_sep_proba5_df.sep5yr)

In [ ]:
off4f

In [ ]:
off5 = find_max_mcc_value(y5,eval_sep_proba5_df.sep5yr)

In [ ]:
plot_conf_matrix(y4,map(np.int,eval_sep_proba4_df.sep4yr+off4),normed=False)
#plt.title('4 year')
print "4 year"
print sum(map(np.int, eval_sep_proba4_df.sep4yr+off4))

In [ ]:
(sep_df.sep4yr+off4).values.astype(int).sum()

### 3 year

In [ ]:
Xe3,y3 =adjust_eval_by_x_years(em2eval_df,3,columns_for_modeling)

print "______________"
#MODEL_BASE_PATH = '/home/hrsepara/work/model/'
#MODEL_BASE_PATH = 'C:\\Users\\kesj\\code\\hrsep1\\model_1\\'
#pred_cases = ['sep/','ret/']
#pred_years = map(np.str,np.arange(1,6))

#eval_sep_proba_df = apply_time_models(Xe5,MODEL_BASE_PATH,pred_cases[0],pred_years)
eval_sep_proba3_df = apply_time_models(Xe3,MODEL_BASE_PATH,pred_cases[0],['3'])

In [ ]:
plot_roc_curve2(y3,eval_sep_proba3_df.sep3yr)

In [ ]:
off3 = find_max_mcc_value(y3,eval_sep_proba3_df.sep3yr)

In [ ]:
off3f = find_max_f1_value(y3,eval_sep_proba3_df.sep3yr)

In [ ]:
plot_conf_matrix(y3,map(np.int,eval_sep_proba3_df.sep3yr+off3),normed=False)
#plt.title('4 year')
print "3 year"
print sum(map(np.int, eval_sep_proba3_df.sep3yr+off3))

In [ ]:
(sep_df.sep3yr+off3).values.astype(int).sum()

### 2 year

In [ ]:
Xe2,y2 =adjust_eval_by_x_years(em2eval_df,2,columns_for_modeling)

print "______________"
#MODEL_BASE_PATH = '/home/hrsepara/work/model/'
#MODEL_BASE_PATH = 'C:\\Users\\kesj\\code\\hrsep1\\model_1\\'
#pred_cases = ['sep/','ret/']
#pred_years = map(np.str,np.arange(1,6))

#eval_sep_proba_df = apply_time_models(Xe5,MODEL_BASE_PATH,pred_cases[0],pred_years)
eval_sep_proba2_df = apply_time_models(Xe2,MODEL_BASE_PATH,pred_cases[0],['2'])

In [ ]:
plot_roc_curve2(y2,eval_sep_proba2_df.sep2yr)

In [ ]:
off2 = find_max_mcc_value(y2,eval_sep_proba2_df.sep2yr)

In [ ]:
off2f = find_max_f1_value(y2,eval_sep_proba2_df.sep2yr)

In [ ]:
plot_conf_matrix(y2,map(np.int,eval_sep_proba2_df.sep2yr+off2),normed=False)
#plt.title('4 year')
print "2 year"
print sum(map(np.int, eval_sep_proba2_df.sep2yr+off2))
print " Number of separations predicted is ",(sep_df.sep2yr+off2).values.astype(int).sum()

### 1 year

In [ ]:
Xe1,y1 =adjust_eval_by_x_years(em2eval_df,1,columns_for_modeling)

print "______________"
#MODEL_BASE_PATH = '/home/hrsepara/work/model/'
#MODEL_BASE_PATH = 'C:\\Users\\kesj\\code\\hrsep1\\model_1\\'
#pred_cases = ['sep/','ret/']
#pred_years = map(np.str,np.arange(1,6))

#eval_sep_proba_df = apply_time_models(Xe5,MODEL_BASE_PATH,pred_cases[0],pred_years)
eval_sep_proba1_df = apply_time_models(Xe1,MODEL_BASE_PATH,pred_cases[0],['1'])

In [ ]:
plot_roc_curve2(y1,eval_sep_proba1_df.sep1yr)

In [ ]:
off1 = find_max_mcc_value(y1,eval_sep_proba1_df.sep1yr)

In [ ]:
off1f = find_max_f1_value(y1,eval_sep_proba1_df.sep1yr)

In [ ]:
plot_conf_matrix(y1,map(np.int,eval_sep_proba1_df.sep1yr+off1),normed=False)
#plt.title('4 year')
print "1 year"
print sum(map(np.int, eval_sep_proba1_df.sep1yr+off1))
print " Number of separations predicted is ",(sep_df.sep1yr+off1).values.astype(int).sum()

In [ ]:
sep_proba_df.hist(bins=30)

In [ ]:
f1_offsets = [off1f,off2f,off3f,off4f,off5f]
f1_offsets

In [ ]:
sep_proba_df.head()

In [ ]:
(sep_proba_df+f1_offsets).values.astype(int).sum(axis=0)
#print " Number of separations predicted is ",(sep_df.sep1yr+off1).values.astype(int).sum()

## Now check for the specific jobcodes & extended functions

In [ ]:
## count how many examples in the current dataframe
[(v,len(sep_df[sep_df.EXT_FUNC_ID_SFI==k])) for k,v in extFuncDict.iteritems()]

In [ ]:
len(current)

In [ ]:
extFuncDict

In [ ]:
sep_pred_cols = ['sep1yr','sep2yr','sep3yr','sep4yr','sep5yr']
ext_val = 71

In [ ]:
print len(sep_df[sep_df.EXT_FUNC_ID_SFI==ext_val])
(sep_df[sep_df.EXT_FUNC_ID_SFI==ext_val][sep_pred_cols]+f1_offsets).values.astype(int).sum(axis=0)

In [ ]:
ext_val = 517
print extFuncDict[ext_val]
print len(sep_df[sep_df.EXT_FUNC_ID_SFI==ext_val])
(sep_df[sep_df.EXT_FUNC_ID_SFI==ext_val][sep_pred_cols]+f1_offsets).values.astype(int).sum(axis=0)

In [ ]:
ext_val = 892
print extFuncDict[ext_val]
print len(sep_df[sep_df.EXT_FUNC_ID_SFI==ext_val])
(sep_df[sep_df.EXT_FUNC_ID_SFI==ext_val][sep_pred_cols]+f1_offsets).values.astype(int).sum(axis=0)

In [ ]:
ext_val = 816
print extFuncDict[ext_val]
print len(sep_df[sep_df.EXT_FUNC_ID_SFI==ext_val])
(sep_df[sep_df.EXT_FUNC_ID_SFI==ext_val][sep_pred_cols]+f1_offsets).values.astype(int).sum(axis=0)

In [ ]:
ext_val = 909
print extFuncDict[ext_val]
print len(sep_df[sep_df.EXT_FUNC_ID_SFI==ext_val])
(sep_df[sep_df.EXT_FUNC_ID_SFI==ext_val][sep_pred_cols]+f1_offsets).values.astype(int).sum(axis=0)

In [ ]:
ext_val = 910
print extFuncDict[ext_val]
print len(sep_df[sep_df.EXT_FUNC_ID_SFI==ext_val])
(sep_df[sep_df.EXT_FUNC_ID_SFI==ext_val][sep_pred_cols]+f1_offsets).values.astype(int).sum(axis=0)

#### Job codes

In [ ]:
invJobCodeDict = {}
invJobCodeDict['CCC Service Rep'] = [6200]
invJobCodeDict['CCC Sales Rep'] = [1876]
invJobCodeDict['Life Customer Service Asst'] = [2638,2639,2641]
invJobCodeDict['Claim Associate-Express'] = [983,984]
invJobCodeDict['Claim Associate-ILR'] = [1009]
invJobCodeDict['Claim Team Manager-ILR'] = [986]
invJobCodeDict['Bank Customer Service Rep'] = [851,852,853]
invJobCodeDict['SFPP Account Rep'] = [4105]#2638,2639,2641]
interest_job_names_list = invJobCodeDict.keys()

In [ ]:
for k in invJobCodeDict.keys():
    print k, sum(sep_df['JOBCODE'].isin(invJobCodeDict[k]))

In [ ]:
jc_val = 'CCC Service Rep'
print invJobCodeDict[jc_val]
print len(sep_df[sep_df.JOBCODE.isin(invJobCodeDict[jc_val])])
pred_vals = (sep_df[sep_df.JOBCODE.isin(invJobCodeDict[jc_val])][sep_pred_cols]+f1_offsets).values.astype(int).sum(axis=0)
for v in pred_vals:
    print "predicted separation count in 1 years = ",v

In [ ]:
for jc_val in invJobCodeDict.keys():
    print "----"
    print invJobCodeDict[jc_val]
    print jc_val
    print len(sep_df[sep_df.JOBCODE.isin(invJobCodeDict[jc_val])])
    pred_vals = (sep_df[sep_df.JOBCODE.isin(invJobCodeDict[jc_val])][sep_pred_cols]+f1_offsets).values.astype(int).sum(axis=0)
    for v in pred_vals:
        print "predicted separation count in 1 years = ",v

## now check on the rates of retirements for each of the coming years
* calculate the total number of retirements and rate of retirement.
    * option 1: sum the probabilities
    * option 2: apply a threshold and sum in binary way.
    

#### Use the survival model

In [ ]:
def apply_survival_models(df,MDL_BASE_PATH,pkl_name,pred_years):

    import pickle
    import pandas as pd
    mdl_name = MDL_BASE_PATH+pkl_name


    try:
        stored_mdl = pickle.load(open(mdl_name,'rb'))
        #stored_mdl = jl.load(mdl_name)

        sf_pred = stored_mdl.predict_survival_function(df)
        sf_pred.index.name='years'
        sf_pred.reset_index(inplace=True)
        pred_proba = get_survival_prediction(df,sf_pred,yr_vals=pred_years)

    except IOError:
        print "requested pickle file not found {0}, using default parameters.".format(mdl_name)
        # TODO create default predictions from simple Survival model for retirement.
        pred_proba = pd.DataFrame()

    return pred_proba


#Define a function to calculate the survival prediction from a pickled model
def get_survival_prediction(edf,psf, age_col='Age_years',yr_vals=[1,2,3,4,5]):
    from scipy import interpolate
    import pandas as pd
    import numpy as np

    pred_col_names = ['proba_'+str(y) for y in yr_vals]
    df = pd.DataFrame(index=edf.index, columns=pred_col_names, dtype=float)
    print pred_col_names
    for employee_id,age in enumerate(edf['Age_years']):
        if employee_id % 5000 ==0 :
            print employee_id, age

        prob_vals =np.ones((len(yr_vals),))
        if age >= 50.0: # only deal with over 50
            for idx, year in enumerate(yr_vals):
                ck_yr = age+year
                try:
                    prior_idx = np.where(psf.years < ck_yr)[0][-1]
                    posterior_idx = np.where(psf.years > ck_yr)[0][0]

                    x = [psf.ix[prior_idx]['years'], psf.ix[posterior_idx]['years']]
                    y = [psf.ix[prior_idx][employee_id+1], psf.ix[posterior_idx][employee_id+1]]
                    # there is an extra column in my processing
                    # now interpolate these
                    y_interp = interpolate.interp1d(x,y)
                    proba = y_interp(ck_yr)
                except IndexError:
                    proba = psf.ix[np.where(psf.years == ck_yr)[0]][employee_id+1]

                prob_vals[idx] = proba

        df.loc[edf.index[employee_id]]=prob_vals
    df.fillna(1.0,inplace=True)  # fill in missing values with ones
    df[df<0] = 0.0  # replace negative probabilities with zero

    return 1-df  # convert to probability of retirement


In [ ]:
cover50 = employee_df[employee_df.Age_years>=50].copy()
len(cover50)

In [ ]:
cover50['hire_age'] = current[current.Age_years>=50].hire_age

In [ ]:
cover50.columns

In [ ]:
np.arange(1,6)

In [ ]:
sfret_pred = apply_survival_models(cover50,MODEL_BASE_PATH,'retirement_sfA.pkl',np.arange(1,6))

In [ ]:
sfret_pred.shape

## Need to develop an independent assessment using the evaluation sets above


In [ ]:
over50eval = em2eval_df[em2eval.Age_years>=50].copy()
len(over50eval)
over50eval['hire_age'] = (over50eval['hire_tstmp']-over50eval['birth_tstmp'])/np.timedelta64(1,'Y')
over50eval['retired']=em2eval[em2eval.Age_years>=50]['retired']
#Xe5,y5 =adjust_eval_by_x_years(em2eval_df,5,columns_for_modeling)

In [ ]:
len(over50eval)

In [ ]:
over50eval.retired.value_counts()

In [ ]:
pd.crosstab(over50eval.retired,over50eval.status)

In [ ]:
## get rid of those that are status==1 and retired == 0 --> just classifying on retired or not
over50eval = over50eval[~((over50eval.retired==0)&(over50eval.status==1))]
len(over50eval)

In [ ]:
#cover50.columns
sfret_pred_eval_set = apply_survival_models(over50eval[cover50.columns],MODEL_BASE_PATH,'retirement_sfA.pkl',np.arange(1,6))

## now assess the accuracy for each year based upon this evaluation set

In [ ]:
sfret_pred_eval_set.head()

In [ ]:
sfret_pred_eval_set['retired']=over50eval['retired']
sfret_pred_eval_set.head()

In [ ]:
sfret_pred_eval_set['term_tstmp'] = em2eval.ix[sfret_pred_eval_set.index]['term_tstmp']
sfret_pred_eval_set[sfret_pred_eval_set['term_tstmp']!='2015-01-01']

In [ ]:
sfret_pred_eval_set.sum()

In [ ]:

ret_off1f = find_max_f1_value(sfret_pred_eval_set.retired,sfret_pred_eval_set.proba_1)

In [ ]:

ret_off2f = find_max_f1_value(sfret_pred_eval_set.retired,sfret_pred_eval_set.proba_2)

In [ ]:

ret_off3f = find_max_f1_value(sfret_pred_eval_set.retired,sfret_pred_eval_set.proba_3)

In [ ]:

ret_off4f = find_max_f1_value(sfret_pred_eval_set.retired,sfret_pred_eval_set.proba_4)

In [ ]:

ret_off5f = find_max_f1_value(sfret_pred_eval_set.retired,sfret_pred_eval_set.proba_5)

In [ ]:
plot_conf_matrix(sfret_pred_eval_set.retired,map(np.int,sfret_pred_eval_set.proba_1+0.94),normed=False)

In [ ]:
ret_offsets = [ret_off1f,ret_off2f,ret_off3f,ret_off4f,ret_off5f]


In [ ]:
(sfret_pred_eval_set[['proba_1','proba_2','proba_3','proba_4','proba_5']]+ret_offsets).values.astype(int).sum(axis=0)

### trying retirement directly

In [ ]:
import pickle
retirement_mdl = pickle.load(open(MODEL_BASE_PATH+'retirement_sfA.pkl','rb'))

In [ ]:
retire_pred = retirement_mdl.predict_expectation(over50eval[cover50.columns])
retire_pred

In [ ]:
retire_pred.rename(columns={0:'pred_retire_age'},inplace=True)

In [ ]:
retire_pred[['Age_years','retired']]=over50eval[['Age_years','retired']]

In [ ]:
actual_retired = retire_pred[retire_pred.retired==1][['pred_retire_age','Age_years']].diff(axis=1)['Age_years'].apply(lambda x: np.abs(x))#.apply(lambda x: x[0]-x[1] ,row=1)# 0 else )

In [ ]:
actual_notretired = retire_pred[retire_pred.retired==0][['pred_retire_age','Age_years']].diff(axis=1)['Age_years'].apply(lambda x: np.abs(x))

In [ ]:
print len(actual_retired)
print "\t TP\t\t FN"
for i in [1.5,2.5,3.5,4.5,5.5]:
    print i, sum(actual_retired<=i), sum(actual_retired>=i)


In [ ]:
print len(actual_notretired)
print "\t TN\t\t FP"
for i in [1.5,2.5,3.5,4.5,5.5]:
    print i, sum(actual_notretired>i), sum(actual_notretired<=i)


## More assessment of the Survival-based models
1. load the survival model for separation
2. look at confidence intervals and how to join

In [ ]:
os.chdir('/home/kesj/work/hrsepara/proc')


In [ ]:
sep_cf1 = pickle.load(open('separation_sf1.pkl','rb'))

In [ ]:
nr_em = em2002[em2002.retired== 0].copy()
nr_em.shape

In [ ]:
# break into evaluation and build sets
print "Starting with subset of {0} employees.".format(len(nr_em))
eval_fraction = 0.20
ne2build, ne2eval = train_test_split(nr_em,test_size=eval_fraction,random_state = 60333)
print "Evaluation set has {0} employees; training set has {1} employees.".format(len(ne2eval),len(ne2build))

In [ ]:
os.chdir(repodir)
import bear.bear as br
os.chdir('/home/kesj/work/hrsepara/proc')

In [ ]:
missing_cols = br.get_columns_with_nulls(ne2build)

In [ ]:
loc_descriptions = em2002.LOC_TYPE_DESCR_SFI.unique().tolist()
short_loc_descript = {}
for loc_name in loc_descriptions:
    if loc_name.endswith(')'):
        short_name = loc_name[-4:-1]
        if short_name == 'NCE':
            short_loc_descript[loc_name]='TELE'
        else:
            short_loc_descript[loc_name]=short_name
    elif loc_name.startswith('TELE'):
        short_loc_descript[loc_name]='TELE'
    elif loc_name.endswith('SIU'):
        short_loc_descript[loc_name]='SIU'
    elif loc_name.startswith('OPERATIONS'):
        short_loc_descript[loc_name]='OC'
    elif loc_name.startswith('CORP'):
        short_loc_descript[loc_name]='CORP'
    elif loc_name.endswith('CENTER'):
        short_loc_descript[loc_name]='MC' # marketing center
    elif loc_name.endswith('AFO'):
        short_loc_descript[loc_name]='AFO'
    elif loc_name.endswith('OFFICE'):
        if loc_name.startswith('AGENT'):
            short_loc_descript[loc_name]='AO' # agent office
        elif loc_name.startswith('REGION'):
            short_loc_descript[loc_name]='RO' # regional office
        elif loc_name.startswith('CATA'):
            short_loc_descript[loc_name]='CAT' # catastrophe office
    elif loc_name.startswith('EST'):
        short_loc_descript[loc_name]='EST' # estimating station
    elif loc_name.startswith('ADM'):
        short_loc_descript[loc_name]='ADM' # estimating station
    elif loc_name.startswith('WARE'):
        short_loc_descript[loc_name]='WARE' # estimating station
    elif loc_name.startswith('VAC'):
        short_loc_descript[loc_name]='VAC' # estimating station
    elif loc_name.startswith('MULT-USE'):
        short_loc_descript[loc_name]='MF' # estimating station
    elif loc_name.startswith('MULT-REG'):
        short_loc_descript[loc_name]='MRSF' # estimating station
    else: 
        short_loc_descript[loc_name]='UNK'

print len(short_loc_descript.keys()), len(set(short_loc_descript.values()))

In [ ]:
my_label_encode_cols = ['GRADE','JOB_FAMILY','LOC_STATE']
cat_cols_to_dummy = ['EMPL_TYPE','FULL_PART_TIME','SHIFT','EMPL_CLASS','EEO1CODE']

In [ ]:
from sklearn.pipeline import Pipeline
#cate_list = br.get_categorical(X)
# need to break the categorical into types
pipeS = Pipeline([#("null", br.RemoveAllNull()),  
                 #("label_encode",br.LabelEncodeColumn(my_label_encode_cols)),
                 #("dummy_encode", br.DummyEncodeColumn(cat_cols_to_dummy)),
                 #("survival_encode",SurvivalEncodeColumn(my_label_encode_cols[:5],method='median')),
                 #("cat", br.ConvertCategorical(cate_list, rows_to_scan=0.3)),
                 ("fixout_min",br.FixNumericOutlier(columns_to_fix=['SAL1','MIN_RT_ANNUAL','MERIT1','PERF1'],
                                                    criteria_coef=('percentile',2),
                                                   method='lower',fill_with='nearest_value')),
                  ("fixout_max",br.FixNumericOutlier(columns_to_fix=['MAX_RT_ANNUAL'],criteria_coef=('percentile',2),
                                                   method='both',fill_with='nearest_value')) ])#,
                 #("fill_missingzero",FillMissingValue(columns_to_fix=['BOX1','FUNC_CNT','EXTFUNC_CNT','TOTAL_RPT_CNT','DIRECT_RPT_CNT'],fill_value=0)),
                 #("imp_mode",br.ImputeData(columns_to_impute=missing_cols,rows_to_scan=0.8))])
                 #("imp_mean", br.ImputeData(columns_to_impute='auto', rows_to_scan=0.8))])
                 #("redu", br.DimensionReduction(rows_to_scan=0.3, n_components=10))])
#pipe.fit(X, y).transform(testX)[0:30]
#
#Xft = pipe.fit_transform(X)

In [ ]:
sep_surv_cols1 = ['status','Age_years','Tenure_years','SAL1','MERIT1','PERF1','BOX1','SEX','HAVE_INS','HAVE_DEP',
                  'DEPTCNT1','EXTFUNC_CNT','FUNC_CNT','REMOTE']

In [ ]:
ne2b = pipeS.fit_transform(ne2build[sep_surv_cols1],ne2build.status)
ne2e = pipeS.fit(ne2build[sep_surv_cols1],ne2build.status).transform(ne2eval[sep_surv_cols1])

In [ ]:
ne2e_psf = sep_cf1.predict_survival_function(ne2e)
ne2e_psf.index.name = 'years'
ne2e_psf.reset_index(inplace=True)
ne2b_psf = sep_cf1.predict_survival_function(ne2b)
ne2b_psf.index.name = 'years'
ne2b_psf.reset_index(inplace=True)

In [ ]:
plt.plot(ne2b_psf.years,ne2b_psf[53117])

In [ ]:
ne2b_expect = sep_cf1.predict_expectation(ne2b)
ne2b_med = sep_cf1.predict_median(ne2b)


In [ ]:
ne2b_uci = sep_cf1.predict_percentile(ne2b,p=0.95)
ne2b_uci.head()

In [ ]:
ne2b_lci = sep_cf1.predict_percentile(ne2b,p=0.05)

In [ ]:
ne2b_pred = pd.concat([ne2b.Tenure_years,ne2b_expect,ne2b_lci,ne2b_med,ne2b_uci],axis=1)
ne2b_pred.shape

In [ ]:
ne2b_pred.head()

In [ ]:
mx_tenure = nr_em.Tenure_years.max()
mx_tenure

In [ ]:
ne2b_pred.replace({ np.inf: mx_tenure},inplace=True) #].any()

In [ ]:
ne2b_pred['status']=ne2b['status']

In [ ]:
sns.distplot(ne2b_pred[0]-ne2b_pred[0.5])

In [ ]:
sns.distplot(ne2b_pred[ne2b_pred.status==0][0]-ne2b_pred[ne2b_pred.status==0][0.5],label='current')
sns.distplot(ne2b_pred[ne2b_pred.status==1][0]-ne2b_pred[ne2b_pred.status==1][0.5],label='former')
plt.legend()

In [ ]:
plt.plot(ne2b_psf.years,ne2b_psf[37240])

In [ ]:
ne2b_pred['med_diff'] = ne2b_pred[0.5] - ne2b_pred['Tenure_years']
ne2b_pred['p05_diff'] = ne2b_pred[0.05] - ne2b_pred['Tenure_years']
ne2b_pred['p95_diff'] = ne2b_pred[0.95] - ne2b_pred['Tenure_years']


In [ ]:
take20 = ne2b.head(20).copy()

In [ ]:
sep_cf1.print_summary()

In [ ]:
take20_cum_hazard = sep_cf1.predict_cumulative_hazard(take20)

In [ ]:
take20_cum_hazard.apply(lambda x: np.exp(-1*x)).plot() # this is the survival function

In [ ]:
sep_cf1.baseline_survival_, sep_cf1.hazards_

In [ ]:
sep_cf1.confidence_intervals_

In [ ]:
sep_cf1.hazards_

In [ ]:
sep_cf1.predict_partial_hazard(take20)#.apply(lambda x: np.log(x))

In [ ]:
from lifelines.utils import normalize
normt20 = normalize(take20[sep_cf1.hazards_.columns], sep_cf1._norm_mean.values, sep_cf1._norm_std.values)

In [ ]:
sep_cf1.normalize

In [ ]:
def predict_partial_hazard_ci(mdl,X):
    index = X.index
    if mdl.normalize:
        X = normalize(X,mdl._norm_mean.values, mdl._norm_std.values)
    df = pd.DataFrame(np.exp(np.dot(X,mdl.confidence_intervals.T)),index=index)
    df.columns.names = ['upperbound','lowerbound']

In [ ]:
if mdl.normalize:
    mdl._n
sep_cf1.normalize


In [ ]:
dtemp = pd.DataFrame(np.exp(np.dot(normt20,sep_cf1.confidence_intervals_.T)),index=take20.index)
dtemp.rename(columns = {0: 'upperbound',1: 'lowerbound'},inplace=True)
dtemp

In [ ]:
#pd.DataFrame(-np.dot(np.log(s_0),
sep_cf1.predict_partial_hazard(take20).T#),index=s_0.index,columns=col)

In [ ]:
pd.DataFrame(dtemp.upperbound).T

In [ ]:
# now caluculate the cumulative_hazard from this:
s_0 = sep_cf1.baseline_survival_
col = dtemp.index.tolist()
dtemp_up = pd.DataFrame(-np.dot(np.log(s_0),pd.DataFrame(dtemp['upperbound']).T),index=s_0.index,columns=col)
dtemp_low = pd.DataFrame(-np.dot(np.log(s_0),pd.DataFrame(dtemp['lowerbound']).T),index=s_0.index,columns=col)
dtemp_up.head()

In [ ]:
t20_up = dtemp_up.apply(lambda x: np.exp(-1*x))
t20_lo = dtemp_low.apply(lambda x: np.exp(-1*x))
t20_psf = sep_cf1.predict_survival_function(take20)


In [ ]:
t20_psf[53117].plot()
t20_up[53117].plot()
t20_lo[53117].plot()

In [ ]:
take20.iloc[0].Tenure_years+1

In [ ]:
t20_up[53117].ix[9.34:9.356]

In [ ]:
pd.DataFrame(np.exp(np.dot(normt20,sep_cf1.hazards_.T)),index=take20.index)

In [ ]:
#np.exp(np.dot(
take20[sep_cf1.hazards_.columns]#, sep_cf1.hazards_.T))

In [ ]:
ne2b_pred.head()

In [ ]:
take20.Tenure_years

In [ ]:
take20.shape

In [ ]:
def get_relevant_baseline_sf(mdl,df,yr_range = [0,1,2,3,4,5],base_time_col='Tenure_years'):
    individuals = df.index.tolist()
    for indiv in individuals:
        my_times = np.array([df.loc[indiv,base_time_col]+year for year in yr_range])
        print indiv, my_times
        # now for these I want to calculate the forecasted survival probability
    
    

In [ ]:
y_interp = interpolate.interp1d(x,y)
                    proba = y_interp(ck_yr)

In [ ]:
get_relevant_baseline_sf(sep_cf1,take20)

In [ ]:
normalize(take20[sep_cf1.hazards_.columns], sep_cf1._norm_mean.values, sep_cf1._norm_std.values)

## let me just apply this model to the current set

In [ ]:
current = em2002[em2002.status == 0].copy()

In [ ]:
len(current)

In [ ]:
sep_cf1.baseline_survival_.plot()

In [ ]:
curr = pipeS.fit(ne2build[sep_surv_cols1],ne2build.status).transform(current[sep_surv_cols1])

In [ ]:
curr_psf = sep_cf1.predict_survival_function(curr)
curr_psf.index.name = 'years'
curr_psf.reset_index(inplace=True)

In [ ]:
def get_survival_prediction(edf,psf, time_col='Tenure_years', yr_vals=[1,2,3,4,5],restrict_time=None):
    from scipy import interpolate
    import pandas as pd
    import numpy as np

    pred_col_names = ['proba_'+str(y) for y in yr_vals]
    df = pd.DataFrame(index=edf.index, columns=pred_col_names, dtype=float)
    print pred_col_names
    employee_idx = edf.index.tolist()
    emp_cols = edf.columns.tolist()
    tcol_idx = emp_cols.index(time_col)+1
    age_idx = emp_cols.index('Age_years')+1
    #define the maximum survival_time_range
    mx_surv_time = psf.years.max()
    if restrict_time is None:
            restrict_time = 0.0
            
    for employee in edf.itertuples():
        my_index = employee[0]
        my_age = employee[age_idx]
        my_time = employee[tcol_idx]
        #print my_index,my_age, my_time
        prob_vals = np.ones(len(yr_vals))
        if my_index % 5000 ==0 :
            print my_index, my_age, my_time
        
        
        if my_age >= restrict_time: # only deal with over 50 (restrict_time = 50.0)
            for idx, year in enumerate(yr_vals):
                ck_yr = my_time+year
                if ck_yr>=mx_surv_time:
                    proba = psf.iloc[-1].loc[my_index]
                else:
                    try:
                        prior_idx = np.where(psf.years < ck_yr)[0][-1]
                        posterior_idx = np.where(psf.years > ck_yr)[0][0]

                        x = [psf.loc[prior_idx]['years'], psf.loc[posterior_idx]['years']]
                        y = [psf.loc[prior_idx][my_index], psf.loc[posterior_idx][my_index]]
                            
                        # now interpolate these
                        y_interp = interpolate.interp1d(x,y)
                        proba = y_interp(ck_yr)
                    
                    except IndexError:
                        #print "\t IE", proba, idx,my_index
                        proba = psf.loc[np.where(psf.years == ck_yr)[0]][my_index]

                prob_vals[idx] = proba
        
        df.loc[my_index]=prob_vals
    
    df.fillna(1.0,inplace=True)  # fill in missing values with ones
    df[df<0] = 0.0  # replace negative probabilities with zero

    return 1-df  # convert to probability of separation or retirement

In [ ]:
curr_sep_pred1 = get_survival_prediction(curr,curr_psf,'Tenure_years')

In [ ]:
curr_sep_pred1.head()

In [ ]:
sep_cf1.print_summary()

In [ ]:
len(curr)

In [ ]:
(curr_sep_pred1+0.1).values.astype(int).sum(axis=0)

In [ ]:
curr_sep_pred1.sum(axis=0)

In [ ]:
#sep_def_off = [0.1,0.3,0.5,0.7,0.9]
sep_def_off = [0.5,0.5,0.5,0.5,0.5]
(curr_sep_pred1+sep_def_off).values.astype(int).sum(axis=0)

In [ ]:
(curr_sep_pred1+sep_def_off-0.1).values.astype(int).sum(axis=0)

In [ ]:
(curr_sep_pred1+sep_def_off+0.1).values.astype(int).sum(axis=0)

In [ ]:
simple_rates = pd.DataFrame(sep_proba_df.sum()/float(len(sep_proba_df)))
simple_rates.index = ['yr1','yr2','yr3','yr4','yr5']
simple_rates.columns = ['Separation']
simple_rates['Annual Separation'] = simple_rates.diff()
simple_rates.loc['yr1','Annual Separation']= simple_rates.loc['yr1','Separation']

In [ ]:
tmp_rates = pd.DataFrame(ret_proba_df.sum()/float(len(ret_proba_df)))
tmp_rates.index = ['yr1','yr2','yr3','yr4','yr5']
tmp_rates.columns = ['Retirement']
tmp_rates['Annual Retirement'] = tmp_rates.diff()
tmp_rates.loc['yr1','Annual Retirement']= tmp_rates.loc['yr1','Retirement']
simple_rates = pd.concat([simple_rates,tmp_rates],axis=1)
simple_rates

## what were historical rates?

In [ ]:
### load in the federal unemployment data
unempl_raw = pd.read_csv('../eda/us_unemployment_monthly_seas.csv')
unempl_raw.head()
### convert to useable format
unempl = pd.melt(unempl_raw, id_vars='Year')
unempl.sort('Year', inplace=True)
unempl['date'] = unempl[['Year','variable']].apply(lambda x: pd.to_datetime(x[1]+str(x[0]),format='%b%Y'),axis=1)
unempl.sort('date',inplace=True)
unempl.head()

In [ ]:
plt.plot(unempl.date,unempl.value)

In [ ]:
unempl.groupby('Year')['value'].mean()

## calculate the historical rates of retirement and separation

In [ ]:
em2002[['hire_tstmp','term_tstmp']] = em2002[['HIRE_DT','TERMINATION_DT']].applymap(lambda x: pd.to_datetime(datetime.datetime.strptime(x,'%d%b%Y')))


In [ ]:
monthly_range_terminations = pd.date_range(em2002.term_tstmp.min(),em2002.term_tstmp.max(),freq='M')
print len(monthly_range_terminations)
monthly_range_terminations

In [ ]:
monthly_range_terminations[0]-1

In [ ]:
len(em2002[(em2002.term_tstmp <= monthly_range_terminations[1]) & (em2002.retired == 0) & (em2002.term_tstmp > monthly_range_terminations[0])])

In [ ]:
nactive = []
nsep2 = []
nhired = []
nretired = []
prev_date = monthly_range_terminations[0]-1
#nsep.append(0.0)
for idx, my_date in enumerate(monthly_range_terminations):
    nactive.append( len(em2002[(em2002.hire_tstmp < my_date) &(em2002.term_tstmp > my_date)]))
    if idx == 0:
        nsep2.append(len(em2002[(em2002.term_tstmp <= my_date) & (em2002.hire_tstmp < my_date)]))
        nretired.append(len(em2002[(em2002.term_tstmp <= my_date) & (em2002.hire_tstmp < my_date) & (em2002.retired==1)]))
        nhired.append( len(em2002[(em2002.hire_tstmp <= my_date) &(em2002.hire_tstmp > prev_date)]))
    else:
        nsep2.append(len(em2002[(em2002.term_tstmp <= my_date) & (em2002.hire_tstmp < my_date) &( em2002.term_tstmp > monthly_range_terminations[idx-1])]))
        nretired.append(len(em2002[(em2002.term_tstmp <= my_date) & (em2002.hire_tstmp < my_date) & (em2002.retired==1) &( em2002.term_tstmp > monthly_range_terminations[idx-1])]) )
        nhired.append( len(em2002[(em2002.hire_tstmp <= my_date) &(em2002.hire_tstmp > monthly_range_terminations[idx-1])]))
        

In [ ]:
workforce= pd.DataFrame(data =nactive,index=monthly_range_terminations,columns=['Active'])
delta = workforce.diff()
workforce['netChange'] = delta
workforce['AllSeparations'] = nsep2
workforce['Hires'] = nhired
workforce['Retirements'] = nretired
workforce.head()

In [ ]:
plt.plot(monthly_range_terminations,nactive)
plt.xlabel('Date')
plt.ylabel('Worforce Counts')

In [ ]:
annualwf = pd.DataFrame()
annualwf = workforce[['AllSeparations','Hires','Retirements']].resample('A',how='sum')#.plot()
annualwf['Active'] =workforce['Active'].resample('A',how='last')
annualwf.head()

In [ ]:
annualwf['Separations']=annualwf['AllSeparations']-annualwf['Retirements']
annualwf['SepRate']=annualwf['Separations']/annualwf['Active']
annualwf['RetRate']=annualwf['Separations']/annualwf['Active']


In [ ]:
(annualwf.AllSeparations/annualwf.Active).plot()

In [ ]:
## Feb. 10, 2016
### trying a survival model for separation (omitting the known retired elements)

In [ ]:
X20 = take20['Tenure_years'].values

In [ ]:
[X20[0]+tp for tp in [0,1,2,3]]

In [ ]:
sep_cf1.durations.name